In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/Users/Kelly/Documents/Documents/CS 234/week-14-day-24-tutorials-kellyc721/my_data/covid_attributes_categories.csv", encoding="utf-8")

In [3]:
df.head()

,QID,label,description,instance_of,subclass_of,label_words,desc_words,instance_words,subclass_words,candidate_categories
0,Q84263196,COVID-19,contagious disease caused by SARS-CoV-2,Q609748,Q2633267,['covid'],"['contagious', 'disease', 'caused', 'sarscov']",['q'],['q'],"['caused', 'contagious', 'sarscov', 'disease',..."
1,Q87866152,COVID-19 pandemic lockdown,states of emergency and regimes of exception d...,curfew,lockdown,"['covid', 'pandemic', 'lockdown']","['states', 'emergency', 'regimes', 'exception'...",['curfew'],['lockdown'],"['curfew', 'states', 'emergency', 'regimes', '..."
2,Q85173778,COVID-19 misinformation,false information about the pandemic caused by...,conspiracy theory,misinformation,"['covid', 'misinformation']","['false', 'information', 'pandemic', 'caused',...","['conspiracy', 'theory']",['misinformation'],"['misinformation', 'coronavirus', 'false', 'co..."
3,Q105334706,COVID-19 misinformation by China,misinformation related to COVID-19 by China,NaN,COVID-19 pandemic misinformation by governments,"['covid', 'misinformation', 'china']","['misinformation', 'related', 'covid', 'china']",[],"['covid', 'pandemic', 'misinformation', 'gover...","['misinformation', 'china', 'governments', 're..."
4,Q104821676,COVID-19 pandemic misinformation by governments,false or misleading information related to the...,NaN,COVID-19 misinformation,"['covid', 'pandemic', 'misinformation', 'gover...","['false', 'misleading', 'information', 'relate...",[],"['covid', 'misinformation']","['misinformation', 'misleading', 'promoted', '..."


In [4]:
df.to_csv("data/covid_attributes_categories.csv", index=False)

In [5]:
candidate_categories = [
    "misinformation",
    "vaccine",
    "treatment",
    "lockdown",
    "human",
    "government",
    "facility",
    "response",
    "variant",
    "societal impact",
    "timeline",
    "disease"
]

In [6]:
df["full_text"] = (
    df["label"].astype(str) + " " +
    df["description"].astype(str) + " " +
    df["instance_of"].astype(str) + " " +
    df["subclass_of"].astype(str)
)

In [ ]:
pip install transformers

In [ ]:
pip install torch

In [ ]:
!pip install torch --index-url https://download.pytorch.org/whl/cpu

In [ ]:
!pip install protobuf

In [7]:
from transformers import pipeline
import torch
from tqdm.notebook import tqdm

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
#MODEL_NAME = "joeddav/xlm-roberta-large-xnli"
MODEL_NAME = "facebook/bart-large-mnli"
DEVICE = 0 if torch.cuda.is_available() else -1

print(f"Loading model: {MODEL_NAME} on device: {'GPU' if DEVICE == 0 else 'CPU'}")

# pipeline is a function from HuggingFace's transformers library
classifier = pipeline(
    "zero-shot-classification",
    model=MODEL_NAME,
    device=DEVICE,
    batch_size=32
)

Loading model: facebook/bart-large-mnli on device: CPU


Device set to use cpu


In [11]:
results = classifier(
        df["full_text"].to_list(),
        candidate_labels=candidate_categories,
        multi_label=False
    )

results

[{'sequence': 'COVID-19 contagious disease caused by SARS-CoV-2 Q609748 Q2633267',
  'labels': ['disease',
   'variant',
   'timeline',
   'response',
   'facility',
   'lockdown',
   'societal impact',
   'human',
   'misinformation',
   'treatment',
   'government',
   'vaccine'],
  'scores': [0.6040393710136414,
   0.22848287224769592,
   0.03840480372309685,
   0.03145045042037964,
   0.025760890915989876,
   0.020366491749882698,
   0.019359642639756203,
   0.012155679054558277,
   0.006817679852247238,
   0.0052108378149569035,
   0.005197469610720873,
   0.002753801643848419]},
 {'sequence': 'COVID-19 pandemic lockdown states of emergency and regimes of exception decreed by COVID-19 curfew lockdown',
  'labels': ['lockdown',
   'disease',
   'variant',
   'societal impact',
   'government',
   'response',
   'timeline',
   'facility',
   'human',
   'vaccine',
   'treatment',
   'misinformation'],
  'scores': [0.369249552488327,
   0.31767433881759644,
   0.16975434124469757,
  

In [14]:
import json
with open("data/categories_classification_results.json", "w") as f:
  json.dump(results, f)

In [15]:
def classify(text):
    result = classifier(
        text,
        candidate_labels=candidate_categories,
        multi_label=False
    )
    return result["labels"][0]

In [49]:
df["predicted_categories"] = df["full_text"].apply(classify)

In [52]:
df.drop(columns=["description", "instance_of", "subclass_of", "label_words", "desc_words", "instance_words", "subclass_words"], inplace=True)

In [53]:
df.head(10)

,QID,label,candidate_categories,full_text,predicted_categories
0,Q84263196,COVID-19,"['caused', 'contagious', 'sarscov', 'disease',...",COVID-19 contagious disease caused by SARS-CoV...,disease
1,Q87866152,COVID-19 pandemic lockdown,"['curfew', 'states', 'emergency', 'regimes', '...",COVID-19 pandemic lockdown states of emergency...,lockdown
2,Q85173778,COVID-19 misinformation,"['misinformation', 'coronavirus', 'false', 'co...",COVID-19 misinformation false information abou...,misinformation
3,Q105334706,COVID-19 misinformation by China,"['misinformation', 'china', 'governments', 're...",COVID-19 misinformation by China misinformatio...,misinformation
4,Q104821676,COVID-19 pandemic misinformation by governments,"['misinformation', 'misleading', 'promoted', '...",COVID-19 pandemic misinformation by government...,misinformation
5,Q104821688,Misinformation related to the COVID-19 pandemi...,"['us', 'misinformation', 'united', 'states', '...",Misinformation related to the COVID-19 pandemi...,misinformation
6,Q81068910,COVID-19 pandemic,"['caused', 'pandemic', 'covid', 'sarscov']",COVID-19 pandemic pandemic caused by SARS-CoV-...,disease
7,Q83741704,COVID-19 pandemic by country and territory,"['wikipedia', 'overview', 'geographic', 'histo...",COVID-19 pandemic by country and territory det...,variant
8,Q106915773,COVID-19 rapid antigen test,"['infection', 'rapid', 'antigen', 'sarscov', '...",COVID-19 rapid antigen test rapid test for SAR...,disease
9,Q87719492,COVID-19 vaccine,"['coronavirus', 'vaccines', 'vaccine', 'sarsco...",COVID-19 vaccine vaccine against SARS-CoV-2 co...,vaccine


Accuracy Evaluation

Generate ground truth data

In [16]:
df_gt_list = df[["label", "candidate_categories"]].copy()
df_gt_list["categories"] = df_gt_list["candidate_categories"].apply(lambda x: x.split(";")[0] if pd.notna(x) else "unknown")
df_gt_list[["label", "categories"]].to_csv("data/ground_truth_list.csv", index=False)

In [ ]:
import ast

df_gt = pd.read_csv("data/ground_truth_list.csv")

def map_keywords_to_category(words):
    if isinstance(words, str):
        words = ast.literal_eval(words)   # convert string list to real list

    words = [w.lower() for w in words]

    # manually added some keywords for each category; ordered from more common terms to specific
    if "misinformation" in words or "false" in words or "fake" in words or "conspiracy" in words:
        return "misinformation" 
    
    if "disease" in words or "pathogen" in words or "symptoms" in words or "infection" in words or "sarscov" in words:
        return "disease"

    if "variant" in words or "lineage" in words or "strain" in words:
        return "variant"
    
    if "vaccine" in words or "vaccination" in words:
        return "vaccine"
    
    if "lockdown" in words or "curfew" in words or "emergency" in words or "restriction" in words or "quarantine" in words:
        return "lockdown"
    
    if "response" in words:
        return "response"
    
    if "timeline" in words or "time" in words:
        return "timeline"
    
    if "drug" in words or "compound" in words or "therapy" in words or "antibody" in words or "chemical" in words or "medication" in words or "treatments" in words:
        return "treatment"

    if "government" in words or "officials" in words or "united" in words:
        return "government"

    if "human" in words or "people" in words:
        return "human"

    if "impact" in words or "social" in words or "racism" in words or "economic" in words:
        return "societal impact"

    if "hospital" in words or "facility" in words:
        return "facility"

    return "other"

df_gt["true_category"] = df_gt["categories"].apply(map_keywords_to_category)

#df_gt[["label","true_category"]].to_csv("data/ground_truth_final.csv", index=False)

df_gt.head()

,label,categories,true_category
0,COVID-19,"['caused', 'contagious', 'sarscov', 'disease',...",disease
1,COVID-19 pandemic lockdown,"['curfew', 'states', 'emergency', 'regimes', '...",lockdown
2,COVID-19 misinformation,"['misinformation', 'coronavirus', 'false', 'co...",misinformation
3,COVID-19 misinformation by China,"['misinformation', 'china', 'governments', 're...",misinformation
4,COVID-19 pandemic misinformation by governments,"['misinformation', 'misleading', 'promoted', '...",misinformation


In [119]:
df_gt = pd.read_csv("data/ground_truth_final.csv")
df_gt.head(20)

,label,true_category
0,COVID-19,disease
1,COVID-19 pandemic lockdown,lockdown
2,COVID-19 misinformation,misinformation
3,COVID-19 misinformation by China,misinformation
4,COVID-19 pandemic misinformation by governments,misinformation
5,Misinformation related to the COVID-19 pandemi...,misinformation
6,COVID-19 pandemic,disease
7,COVID-19 pandemic by country and territory,disease
8,COVID-19 rapid antigen test,disease
9,COVID-19 vaccine,vaccine


In [120]:
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score

def evaluate_zero_shot_multiclass(classification_results, ground_truth):
    """
    Evaluate zero-shot predictions for multi-class classification.
    """

    # extract top predicted label per article
    y_pred = [result["labels"][0] for result in classification_results]

    # load ground truth category column 
    y_true = df_gt["true_category"].to_list()

    # all categories 
    categories = sorted(list(set(y_true)))

    cm = confusion_matrix(y_true, y_pred, labels=categories)

    accuracy = accuracy_score(y_true, y_pred)

    recall = recall_score(y_true, y_pred, average="macro")
    precision = precision_score(y_true, y_pred, average="macro")
    f1 = f1_score(y_true, y_pred, average="macro")

    print("--- Multi-Class Classification Evaluation ---")
    print(f"Labels: {categories}")
    print("Confusion Matrix:\n", cm)
    print("\nAccuracy:", accuracy)
    print("Recall:", recall)
    print("Precision:", precision)
    print("F1-Score:", f1)

In [121]:
evaluate_zero_shot_multiclass(results, df_gt)

--- Multi-Class Classification Evaluation ---
Labels: ['disease', 'facility', 'government', 'human', 'lockdown', 'misinformation', 'other', 'response', 'societal impact', 'timeline', 'treatment', 'vaccine', 'variant']
Confusion Matrix:
 [[184   0   0   1   0   0   0   1   1   9   0   1  17]
 [  0   6   0   0   0   0   0   0   0   0   0   0   2]
 [  4   3   9   0   0   0   0   4   0   3   0   1  11]
 [  0   0   0  28   0   0   0   0   0   1   0   0   0]
 [  2   0   0   0   7   0   0   0   0   2   0   0   2]
 [  0   0   0   0   0  11   0   0   0   0   0   0   0]
 [  4  10   1   1   0   0   0   0   1   6   0   0  34]
 [  1   0   5   0   0   0   0  35   0  16   0   0   0]
 [ 10   0   0   0   0   0   0   2   4   3   0   0  37]
 [  0   0   0   0   0   0   0   0   0  55   0   0   0]
 [  1   0   0   0   0   0   0   1   0   0  11   0  16]
 [  0   0   0   0   0   0   0   0   0   1   0 104   4]
 [  0   0   0   0   0   0   0   0   0   0   0   0  10]]

Accuracy: 0.6793557833089312
Recall: 0.6453720

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [122]:
import json
with open('data/categories_classification_results.json', 'r') as f:
  predicted_category_results = json.load(f)

df_category = pd.DataFrame(predicted_category_results)
df_category.head()

,sequence,labels,scores
0,COVID-19 contagious disease caused by SARS-CoV...,"[disease, variant, timeline, response, facilit...","[0.6040393710136414, 0.22848287224769592, 0.03..."
1,COVID-19 pandemic lockdown states of emergency...,"[lockdown, disease, variant, societal impact, ...","[0.369249552488327, 0.31767433881759644, 0.169..."
2,COVID-19 misinformation false information abou...,"[misinformation, disease, variant, facility, s...","[0.7120131850242615, 0.1394554227590561, 0.071..."
3,COVID-19 misinformation by China misinformatio...,"[misinformation, government, variant, disease,...","[0.6327171325683594, 0.2048257440328598, 0.055..."
4,COVID-19 pandemic misinformation by government...,"[misinformation, government, variant, societal...","[0.5359662771224976, 0.37495142221450806, 0.02..."


In [123]:
df_category['predicted_label'] = df_category['labels'].apply(lambda x: x[0])
df_category.head()

,sequence,labels,scores,predicted_label
0,COVID-19 contagious disease caused by SARS-CoV...,"[disease, variant, timeline, response, facilit...","[0.6040393710136414, 0.22848287224769592, 0.03...",disease
1,COVID-19 pandemic lockdown states of emergency...,"[lockdown, disease, variant, societal impact, ...","[0.369249552488327, 0.31767433881759644, 0.169...",lockdown
2,COVID-19 misinformation false information abou...,"[misinformation, disease, variant, facility, s...","[0.7120131850242615, 0.1394554227590561, 0.071...",misinformation
3,COVID-19 misinformation by China misinformatio...,"[misinformation, government, variant, disease,...","[0.6327171325683594, 0.2048257440328598, 0.055...",misinformation
4,COVID-19 pandemic misinformation by government...,"[misinformation, government, variant, societal...","[0.5359662771224976, 0.37495142221450806, 0.02...",misinformation


In [124]:
df_category['score'] = df_category['scores'].apply(lambda x: x[0])
df_category.head()

,sequence,labels,scores,predicted_label,score
0,COVID-19 contagious disease caused by SARS-CoV...,"[disease, variant, timeline, response, facilit...","[0.6040393710136414, 0.22848287224769592, 0.03...",disease,0.604039
1,COVID-19 pandemic lockdown states of emergency...,"[lockdown, disease, variant, societal impact, ...","[0.369249552488327, 0.31767433881759644, 0.169...",lockdown,0.369250
2,COVID-19 misinformation false information abou...,"[misinformation, disease, variant, facility, s...","[0.7120131850242615, 0.1394554227590561, 0.071...",misinformation,0.712013
3,COVID-19 misinformation by China misinformatio...,"[misinformation, government, variant, disease,...","[0.6327171325683594, 0.2048257440328598, 0.055...",misinformation,0.632717
4,COVID-19 pandemic misinformation by government...,"[misinformation, government, variant, societal...","[0.5359662771224976, 0.37495142221450806, 0.02...",misinformation,0.535966


In [125]:
df_category.drop(columns=['labels', 'scores', 'sequence'], inplace=True)
df_category.head()

,predicted_label,score
0,disease,0.604039
1,lockdown,0.369250
2,misinformation,0.712013
3,misinformation,0.632717
4,misinformation,0.535966


In [126]:
df_category['ground_truth'] = df_gt['true_category']
df_category.insert(loc=0, column="article", value=df_gt['label'])
df_category.head(30)

,article,predicted_label,score,ground_truth
0,COVID-19,disease,0.604039,disease
1,COVID-19 pandemic lockdown,lockdown,0.369250,lockdown
2,COVID-19 misinformation,misinformation,0.712013,misinformation
3,COVID-19 misinformation by China,misinformation,0.632717,misinformation
4,COVID-19 pandemic misinformation by governments,misinformation,0.535966,misinformation
5,Misinformation related to the COVID-19 pandemi...,misinformation,0.801575,misinformation
6,COVID-19 pandemic,disease,0.299496,disease
7,COVID-19 pandemic by country and territory,variant,0.296042,disease
8,COVID-19 rapid antigen test,disease,0.315814,disease
9,COVID-19 vaccine,vaccine,0.583604,vaccine


In [127]:
df_category.to_csv("data/predicted_categories.csv", index=False)